<a href="https://colab.research.google.com/github/fsilva0703/AxpoTrade/blob/main/project01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Install required libraries
!pip install transformers datasets evaluate peft

# Import necessary libraries
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
import numpy as np

# Load the IMDb dataset and take a smaller subset
dataset = load_dataset("imdb")
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))  # 1000 samples for training
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(200))  # 200 samples for testing

# Load the tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=128)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

# Load the pretrained BERT model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Evaluate the original model
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # 1 epoch for faster training
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",  # Disable W&B
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Evaluate the foundation model
print("Evaluating the original model...")
original_results = trainer.evaluate()
print("Original Model Accuracy:", original_results["eval_accuracy"])

# Apply LoRA PEFT
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence classification
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

peft_model = get_peft_model(model, lora_config)

# Train the PEFT model
print("Training the PEFT model...")
trainer.model = peft_model  # Update trainer to use the PEFT model
trainer.train()

# Save the fine-tuned model
peft_model.save_pretrained("./peft-bert-imdb")

# Load the fine-tuned model for evaluation
print("Loading the fine-tuned model...")
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("./peft-bert-imdb", num_labels=2)

trainer.model = fine_tuned_model

# Evaluate the fine-tuned model
print("Evaluating the fine-tuned model...")
fine_tuned_results = trainer.evaluate()
print("Fine-Tuned Model Accuracy:", fine_tuned_results["eval_accuracy"])


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Original Model Accuracy: 0.51
Training the PEFT model...


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,No log,0.690581,0.013100,0.515000


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading the fine-tuned model...
Evaluating the fine-tuned model...


Fine-Tuned Model Accuracy: 0.515


In [ ]:
pip install -U bitsandbytes